In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta

In [2]:
df = pd.read_excel("products.xlsx")
predictions = pd.read_excel("predicted_products.xlsx")
df.shape

(128142, 10)

In [3]:
df.head()

,Base Product,Description,Date,Quantity,Variation Type,Variation Detail,Customisation Complexity,Material,Price,Country
0,12 pencils tall tube,12 pencils tall tube skulls,2009-12-01,28,miscellaneous,miscellaneous: skulls,1,polymer,0.85,United Kingdom
1,12 pencils tall tube,12 pencils tall tube skulls,2009-12-02,25,miscellaneous,miscellaneous: skulls,1,polymer,0.85,United Kingdom
2,12 pencils tall tube,12 pencils tall tube skulls,2009-12-03,28,miscellaneous,miscellaneous: skulls,1,polymer,0.85,United Kingdom
3,12 pencils tall tube,12 pencils tall tube skulls,2009-12-04,79,miscellaneous,miscellaneous: skulls,1,polymer,0.85,United Kingdom
4,12 pencils tall tube,12 pencils tall tube skulls,2009-12-05,12,miscellaneous,miscellaneous: skulls,1,polymer,0.85,United Kingdom


In [4]:
df[df["Date"] == "2011-08-31"]

,Base Product,Description,Date,Quantity,Variation Type,Variation Detail,Customisation Complexity,Material,Price,Country
258,12 pencils tall tube,12 pencils tall tube skulls,2011-08-31,1,miscellaneous,miscellaneous: skulls,1,polymer,1.63,United Kingdom
595,12 pencils tall tube,12 pencils tall tube woodland,2011-08-31,12,miscellaneous,miscellaneous: woodland,1,polymer,0.85,Belgium
872,12 pencils tube,12 pencils tall tube red retrospot,2011-08-31,1,"miscellaneous, size, colour","colour: red, size: tall, miscellaneous: retrospot",3,polymer,1.63,United Kingdom
1343,12 rose peg place settings,12 ivory rose peg place settings,2011-08-31,24,colour,colour: ivory,1,polymer,1.25,United Kingdom
4212,36 pencils tube red,36 pencils tube red retrospot,2011-08-31,1,miscellaneous,miscellaneous: retrospot,1,polymer,2.46,United Kingdom
...,...,...,...,...,...,...,...,...,...,...
123884,washing up gloves,retrospot red washing up gloves,2011-08-31,6,"miscellaneous, colour","colour: red, miscellaneous: retrospot",2,polymer,2.10,Germany
125694,wicker heart,ivory wicker heart large,2011-08-31,5,"size, colour","colour: ivory, size: large",2,wicker,1.65,United Kingdom
125803,wicker star,white wicker star,2011-08-31,2,colour,colour: white,1,wicker,4.13,United Kingdom
127786,wreath decoration with bell,star wreath decoration with bell,2011-08-31,5,miscellaneous,miscellaneous: star,1,polymer,1.25,United Kingdom


In [5]:
### Have not figured out this actual quantity and why it differs from quantity ###
predictions.head()

,Base Product,Description,Date,Quantity,Variation Type,Variation Detail,Customisation Complexity,Material,Price,Country,Actual Quantity,Predicted Quantity
0,12 pencils tall tube,12 pencils tall tube skulls,2011-08-31,1,miscellaneous,miscellaneous: skulls,1,polymer,1.63,United Kingdom,5.0,12.671024
1,12 pencils tall tube,12 pencils tall tube skulls,2011-09-02,8,miscellaneous,miscellaneous: skulls,1,polymer,1.63,United Kingdom,15.0,11.021543
2,12 pencils tall tube,12 pencils tall tube skulls,2011-09-05,13,miscellaneous,miscellaneous: skulls,1,polymer,0.85,United Kingdom,6.0,11.108441
3,12 pencils tall tube,12 pencils tall tube skulls,2011-09-06,12,miscellaneous,miscellaneous: skulls,1,polymer,0.85,United Kingdom,26.0,8.349853
4,12 pencils tall tube,12 pencils tall tube skulls,2011-09-08,12,miscellaneous,miscellaneous: skulls,1,polymer,0.85,United Kingdom,13.0,8.899697


In [7]:
product_dict = df.groupby('Base Product')['Description'].unique().apply(list).to_dict()
print(product_dict)
num_of_products = list(product_dict.keys())
print(len(num_of_products))

{'12 pencils tall tube': ['12 pencils tall tube skulls', '12 pencils tall tube woodland'], '12 pencils tube': ['12 pencils tall tube posy', '12 pencils tall tube red retrospot', '12 pencils tall tube red spotty'], '12 rose peg place settings': ['12 ivory rose peg place settings', '12 pink rose peg place settings', '12 red rose peg place settings'], '2 picture book eggs easter': ['2 picture book eggs easter bunny'], '20 dolly pegs': ['20 dolly pegs retrospot', 'pack 20 dolly pegs'], '200 bendy straws': ['200 bendy skull straws', '200 red white bendy straws'], '3 morris boxed candles': ['3 rose morris boxed candles'], '3 piece mini dots cutlery set': ['blue 3 piece mini dots cutlery set', 'green 3 piece mini dots cutlery set', 'pink 3 piece mini dots cutlery set', 'red 3 piece mini dots cutlery set'], '3 piece polkadot cutlery set': ['blue 3 piece polkadot cutlery set', 'green 3 piece polkadot cutlery set', 'pink 3 piece polkadot cutlery set'], '36 doilies': ['set of 36 doilies spaceboy 

In [8]:
# Function to run one simulation for a given safety stock and reorder point
def run_inventory_simulation(safety_stock, reorder_point, predictions_description_df, mean_demand):
    total_holding_cost = 0
    total_ordering_cost = 0
    total_stockout_cost = 0
    total_units_short = 0
    orders_placed = 0
    service_level = 0
    inventory_level = safety_stock  # Starting inventory level includes safety stock

    num_days = len(predictions_description_df)
    last_date = predictions_description_df.iloc[num_days-1]["Date"]
    
    for i in range(num_days):
        date = predictions_description_df.iloc[i]["Date"]
        daily_demand = max(0, int(predictions_description_df.iloc[i]["Predicted Quantity"]))

        # Check if reorder is needed
        if inventory_level <= reorder_point:
            # Generate a lead time for this order
            lead_time = max(1, int(np.random.normal(mean_lead_time, std_lead_time)))
            orders_placed += 1
            total_ordering_cost += ordering_cost_per_unit * (reorder_point-safety_stock) # Reorder Quantity

            # Add new stock after lead time
            if date + timedelta(days = lead_time) < last_date:
                inventory_level += reorder_point - safety_stock # Reorder Quantity

        # Calculate stockout (if any) and update inventory
        if inventory_level >= daily_demand:
            inventory_level -= daily_demand
        else:
            units_short = daily_demand - inventory_level
            total_stockout_cost += units_short * stockout_cost_per_unit
            total_units_short += units_short
            inventory_level = 0

        # Calculate daily holding cost
        total_holding_cost += inventory_level * holding_cost_per_unit_per_day

    # Calculate service level
    service_level = 1 - (total_units_short / (mean_demand * num_days))

    return {
        "Total Holding Cost": total_holding_cost,
        "Total Ordering Cost": total_ordering_cost,
        "Total Stockout Cost": total_stockout_cost,
        "Total Cost": total_holding_cost + total_ordering_cost + total_stockout_cost,
        "Orders Placed": orders_placed,
        "Service Level": service_level
    }

In [9]:
df["Revenue"] = df["Price"] * df["Quantity"]
annual_inventory_val = df["Revenue"].sum()/25*12 # Data has 25 months
print(0.2*annual_inventory_val) # 20% of Annuval Invetory Value for Holding Costs
print(0.2*annual_inventory_val/365) 

452252.87808
1239.0489810410959


In [10]:
inventory_optimisation = []

## Simulation Hyperparameter ##
# Cost Parameter #
holding_cost_per_unit_per_day = df["Price"].mean() # Cost to hold one unit in inventory for one day (Mean Price of all products in inventory)

for base_product, descriptions in product_dict.items():
    # print("Simulation for Base Product:", base_product)
    
    product_df = df[df['Base Product'] == base_product].copy()
    predictions_df = predictions[predictions["Base Product"] == base_product].copy()
    
    for description in descriptions:
        # print(f"Processing Description: {description} under Base Product: {base_product}")
        description_df = product_df[product_df['Description'] == description].drop(columns=['Description', 'Base Product'])
        predictions_description_df = predictions_df[predictions_df["Description"] == description].drop(columns=['Description', 'Base Product'])
            
        ## Simulation Hyperparams ##
        mean_demand = int(description_df["Quantity"].mean())
        max_lead_time = 9
        mean_lead_time = 5
        std_lead_time = 2   
            
        # Cost parameters #
        ordering_cost_per_unit = description_df["Price"].mean()/120*100 # Fixed order cost per unit (Assuming retail price is 120% of cost price)
        stockout_cost_per_unit = description_df["Price"].mean() # Penalty cost for each unit not met (The retail price basically, since we do not have customer satisfaction values to include)
        
        # Recommended Inventory Control Metrics
        # Safety Stock and Reorder Point Strategy
        # using Safaety Stock = 75% Quantile of Demand During Lead Time * Max Lead Time (due to big outliers and computational resources)
        # using Reorder Point = Mean Demand During Lead time * Mean Lead Time
        ### Also from prev point (Do I use quantity or actual quantity?) ###
        rec_safety_stock = description_df["Quantity"].quantile(q = 0.75).astype(int) * max_lead_time
        rec_reorder_point = mean_demand * mean_lead_time + rec_safety_stock
        
        safety_stock_levels = range(description_df["Quantity"].quantile(q = 0.75).astype(int), rec_safety_stock+100, 100)  # Different safety stock levels to test
            
        ## Collect the results of running the simulation to find the optimal levels
        results = []
        for safety_stock in safety_stock_levels:
            reorder_points = range(description_df["Quantity"].quantile(q = 0.75).astype(int), rec_reorder_point+50, 50)    # Different reorder points to test
            for reorder_point in reorder_points:
                if (reorder_point > safety_stock):
                    result = run_inventory_simulation(safety_stock, reorder_point, predictions_description_df, mean_demand)
                    total_cost = result["Total Cost"]
                    service_level = result["Service Level"]
                    results.append({
                    "Base Product": base_product,
                    "Description": description,
                    "Safety Stock": safety_stock,
                    "Reorder Point": reorder_point,
                    "Total Cost": total_cost,
                    "Service Level": service_level
        })
        df_results = pd.DataFrame(results)
        # print(df_results)
            
        ## Find the optimal Safety Stock Level and Reorder Point for each description in every base product
        best_cost = df_results["Total Cost"].max()
        best_service = df_results["Service Level"].min()
        optimal_safety_stock = df_results.iloc[0]["Safety Stock"]
        optimal_reorder_point = df_results.iloc[0]["Reorder Point"]
            
        for i in range(len(df_results)):
            if (df_results.iloc[i]["Total Cost"] <= best_cost) & (df_results.iloc[i]["Service Level"] >= best_service):
                best_cost = df_results.iloc[i]["Total Cost"]
                best_service = df_results.iloc[i]["Service Level"]
                optimal_safety_stock = df_results.iloc[i]["Safety Stock"]
                optimal_reorder_point = df_results.iloc[i]["Reorder Point"]
        # print(base_product, description, "Safety Stock:", optimal_safety_stock, "Reorder Point:", optimal_reorder_point, best_cost, best_service)
            
        inventory_optimisation.append({
            "Base Product": base_product,
            "Description": description,
            "Safety Stock": optimal_safety_stock,
            "Reorder Point": optimal_reorder_point
        })

inventory_optimisation = pd.DataFrame(inventory_optimisation)
    

In [ ]:
# inventory_optimisation.to_excel("inventory_optimisation.xlsx")

In [11]:
final = []
df2 = predictions.copy(deep=True)
df2["Inventory"] = 0
df2["Predicted Quantity"] = df2["Predicted Quantity"].astype(int)
# print(df2.shape)

for i in range(len(inventory_optimisation)):
    base_product = inventory_optimisation.iloc[i]["Base Product"]
    description = inventory_optimisation.iloc[i]["Description"]
    product_safety_stock = inventory_optimisation.iloc[i]["Safety Stock"]
    product_reorder_point = inventory_optimisation.iloc[i]["Reorder Point"]
    
    inventory_df = df2[df2["Description"] == description].copy()
    # print(inventory_df)
    days = len(inventory_df)
    max_date = inventory_df.iloc[days-1]["Date"]
    product_inventory = product_safety_stock
    
    for i in range(days):
        curr_date = inventory_df.iloc[i]["Date"]
        product_demand = inventory_df.iloc[i]["Predicted Quantity"]  

        # Check if reorder is needed
        if product_inventory <= product_reorder_point:
            # Generate a lead time for this order
            lead_time = max(1, int(np.random.normal(5, 2)))

            # Add new stock after lead time
            if curr_date + timedelta(days = lead_time) < max_date:
                product_inventory += product_reorder_point - product_safety_stock # Reorder Quantity
        
        # Calculate stockout (if any) and update inventory
        if product_inventory >= product_demand:
            product_inventory -= product_demand
        else:
            product_inventory = 0
        final.append({
                "Base Product": base_product,
                "Description": description,
                "Date": curr_date,
                "Inventory": product_inventory
        })
        
df3 = pd.DataFrame(final)
    

In [14]:
df4 = df.merge(df3, on = ["Base Product", "Description", "Date"], how = "left")
print(df.shape, df3.shape, df4.shape)


(128142, 11) (18127, 4) (128142, 12)


In [15]:
print(df4.head())
df4[df4["Inventory"].notna()].shape

           Base Product                  Description       Date  Quantity  \
0  12 pencils tall tube  12 pencils tall tube skulls 2009-12-01        28   
1  12 pencils tall tube  12 pencils tall tube skulls 2009-12-02        25   
2  12 pencils tall tube  12 pencils tall tube skulls 2009-12-03        28   
3  12 pencils tall tube  12 pencils tall tube skulls 2009-12-04        79   
4  12 pencils tall tube  12 pencils tall tube skulls 2009-12-05        12   

  Variation Type       Variation Detail  Customisation Complexity Material  \
0  miscellaneous  miscellaneous: skulls                         1  polymer   
1  miscellaneous  miscellaneous: skulls                         1  polymer   
2  miscellaneous  miscellaneous: skulls                         1  polymer   
3  miscellaneous  miscellaneous: skulls                         1  polymer   
4  miscellaneous  miscellaneous: skulls                         1  polymer   

   Price         Country  Revenue  Inventory  
0   0.85  United King

(18127, 12)

In [ ]:
# df4.to_excel("products with inventory.xlsx")